In [53]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer() #used for stemming the words

import numpy as np
import tflearn # to impliment ANN
import tensorflow as tf
import random

In [54]:
# import our chat-bot intents file(the intent.json file consists of tags(9classes),patterns(27questions) and responses(answers))
import json
with open('.\intent.json') as json_data:
    intents = json.load(json_data)

In [55]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [56]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))


In [57]:
print (len(documents), "documents",documents)
print("########################################################")
print (len(classes), "classes", classes)
print("########################################################")
print (len(words), "unique stemmed words", words)

27 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'), (['What', 'are', 'your', 'hours',

we need to transform it further: from documents of words into tensors of numbers

In [58]:
# creating our training data
training = []
output = []
total_data=[]

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    #print(pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print(output_row)
    total_data.append(output_row)
    training.append([bag, output_row])
    print(training)
print(len(training)) #27 sentences are there.

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0]]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [59]:
import pandas as pd
pd.DataFrame(total_data,columns=classes)

,goodbye,greeting,hours,mopeds,opentoday,payments,rental,thanks,today
0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,1,0


In [60]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
#print(training)

# create train and test lists(:,0 is bag of words, 1 is class)
train_x = list(training[:,0])
train_y = list(training[:,1])
#print(train_x)
print("##############################################")
print(train_y)

##############################################
[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [61]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network(27 sentences i,e; 27 input nodes will be there)len(train_x[0])
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)#if v increase the no of nodes overfitting might occur
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')#9 output layers(9 classes)len(train_y[0])
net = tflearn.regression(net)

In [62]:
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm(adam optimizer))
model.fit(train_x, train_y, n_epoch=3000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Training Step: 5999  | total loss: 0.99415 | time: 0.004s
| Adam | epoch: 3000 | loss: 0.99415 - acc: 0.8213 -- iter: 16/27
Training Step: 6000  | total loss: 0.90545 | time: 0.008s
| Adam | epoch: 3000 | loss: 0.90545 - acc: 0.8392 -- iter: 27/27
--
INFO:tensorflow:C:\Users\Alok\PycharmProjects\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [63]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

# Another notebook for same task

In [64]:
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('.\intent.json') as json_data:
    intents = json.load(json_data)

In [66]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Alok\PycharmProjects\chatbot\model.tflearn


In [67]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [68]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [69]:
p = bow("is your shop open today?", words)
df_data=[]
df_data.append(p)
pd.DataFrame(df_data,columns=words)

,'d,'s,a,acceiv,anyon,ar,bye,can,card,cash,...,thi,to,today,we,what,when,which,work,yo,you
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


# Response Processor

In [70]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    print("results",results)
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [86]:
def response(sentence, userID='123', show_details=False):#show_detais show the context tag like those detais
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [84]:
classify('is your shop open today?')
#so there gives 9 probablities because v have 9 tags, from highest one vll get response

[('opentoday', 0.78227895)]

In [85]:
response('is your shop open today?')
#so there gives 9 probablities because v have 9 tags, from highest one vll get response

Our hours are 9am-9pm every day


In [74]:
response('do you take cash?')

results [0.00161332 0.06211612 0.04596341 0.0012262  0.02410178 0.7521848
 0.00183043 0.0715353  0.03942868]
We accept most major credit cards


In [75]:
response('what kind of mopeds do you rent?')

results [4.74448828e-03 1.95545799e-05 1.12477006e-04 9.69787300e-01
 4.89261607e-03 2.58968868e-07 2.03140862e-02 2.22330755e-06
 1.26955289e-04]
We rent Yamaha, Piaggio and Vespa mopeds


In [76]:
response('Goodbye, see you later')

results [9.8145342e-01 1.0869181e-03 1.4904458e-06 7.2222422e-03 8.4371845e-07
 1.3356924e-07 9.7523564e-03 4.1229013e-04 7.0271803e-05]
Have a nice day


# For storing the context

In [83]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25 #if the probability is greater than 25% responds.
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [78]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: 
                            print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: 
                            print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [79]:
response('today',show_details=True)

In [80]:
classify('we want to rent a moped')

[('rental', 0.87961966)]

In [81]:
response('we want to rent a moped',show_details=True)

context: rentalday
tag: rental
Are you looking to rent today or later this week?


In [82]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Hi there, how can I help?
